In [42]:
import numpy as np
import pandas as pd

In [43]:
Header=["userid","itemid","rating","timestamp"]
df=pd.read_csv("u.data",sep="\t",names=Header)

print(df)


       userid  itemid  rating  timestamp
0         196     242       3  881250949
1         186     302       3  891717742
2          22     377       1  878887116
3         244      51       2  880606923
4         166     346       1  886397596
5         298     474       4  884182806
6         115     265       2  881171488
7         253     465       5  891628467
8         305     451       3  886324817
9           6      86       3  883603013
10         62     257       2  879372434
11        286    1014       5  879781125
12        200     222       5  876042340
13        210      40       3  891035994
14        224      29       3  888104457
15        303     785       3  879485318
16        122     387       5  879270459
17        194     274       2  879539794
18        291    1042       4  874834944
19        234    1184       2  892079237
20        119     392       4  886176814
21        167     486       4  892738452
22        299     144       4  877881320
23        291   

In [44]:
nuser=df.userid.unique().shape[0]
nitem=df.itemid.unique().shape[0]

print("No of users: ",nuser)
print("No of items: ",nitem)

No of users:  943
No of items:  1682


In [45]:
from sklearn.model_selection import train_test_split
train_data,test_data= train_test_split(df, test_size = 0.25, random_state = 0)
print("Train data Shape:" ,train_data.shape)
print("Test data Shape:" ,test_data.shape)


Train data Shape: (75000, 4)
Test data Shape: (25000, 4)


In [46]:
data_matrix=np.zeros((nuser,nitem))
for line in df.itertuples():
    data_matrix[line[1]-1,line[2]-1]=line[3]

In [47]:
train_data_matrix=np.zeros((nuser,nitem))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1,line[2]-1]=line[3]
    
test_data_matrix=np.zeros((nuser,nitem))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1,line[2]-1]=line[3]
print("Train_matrix data Shape:" ,train_data_matrix.shape)
print("Test matrix data Shape:" ,test_data_matrix.shape)

Train_matrix data Shape: (943, 1682)
Test matrix data Shape: (943, 1682)


In [48]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity=pairwise_distances(train_data_matrix,metric="cosine")
item_similarity=pairwise_distances(train_data_matrix.T,metric=)
print("User_similarity_matrix shape:" ,user_similarity.shape)
print("Item_similarity_matrix shape:" ,item_similarity.shape)

User_similarity_matrix shape: (943, 943)
Item_similarity_matrix shape: (1682, 1682)


In [49]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction,ground_truth):
    prediction=prediction[ground_truth.nonzero()].flatten()
    ground_truth=ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,ground_truth))

In [50]:
def prediction_normal(rating,similarity,type="user"):
    if type=="user":
        
        pred=similarity.dot(rating)/np.array([np.abs(similarity).sum(axis=1)]).T
        
    elif type=="item":
        pred=rating.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    
    return pred
item_prediction_normal=prediction_normal(train_data_matrix,item_similarity,type="item")
user_prediction_normal=prediction_normal(train_data_matrix,user_similarity,type="user")



print("User_prediction_normal shape:" ,user_prediction_normal.shape)
print("Item_similarity_normal shape:" ,item_prediction_normal.shape)

print(rmse(user_prediction_normal,test_data_matrix))



print(rmse(item_prediction_normal,test_data_matrix))





User_prediction_normal shape: (943, 1682)
Item_similarity_normal shape: (943, 1682)
3.25862779481473
3.4472573910990616


In [51]:
def prediction_topk(rating,similarity,type="user",k=40):
    pred=np.zeros((nuser,nitem))
    if type=="user":
        for i in range(nuser):
            topkuser=[np.argsort(similarity[:,i])[:-40-1:-1]]
            for j in range(nitem):
                pred[i,j]=similarity[i,:][topkuser].dot(rating[:,j][topkuser])/np.sum(np.array([np.abs(similarity[i,:][topkuser])]).T)
    if type=="item":
        for j in range(nitem):
            topkuser=[np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(nuser):
                pred[i,j]=similarity[j,:][topkuser].dot(rating[i,:][topkuser])/np.sum(np.array([np.abs(similarity[i,:][topkuser])]))       
    return pred





item_prediction_topk=prediction_topk(train_data_matrix,item_similarity,type="item")
user_prediction_topk=prediction_topk(train_data_matrix,user_similarity,type="user")
print("User_prediction__topk shape:" ,user_prediction_topk.shape)
print("Item_similarity_topk shape:" ,item_prediction_topk.shape)

print(rmse(user_prediction_topk,test_data_matrix))



print(rmse(item_prediction_topk,test_data_matrix))




User_prediction__topk shape: (943, 1682)
Item_similarity_topk shape: (943, 1682)
3.611287648859212
3.674005959885108


In [52]:
def prediction_nobias(rating,similarity,type="user"):
    if type=="user":
        mean_rating=rating.mean(axis=1)
        rating_diff=rating-mean_rating[:,np.newaxis]
        pred=mean_rating[:,np.newaxis]+similarity.dot(rating_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
        
    elif type=="item":
        pred=rating.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    
    return pred


item_prediction_nobias=prediction_nobias(train_data_matrix,item_similarity,type="item")
user_prediction_nobias=prediction_nobias(train_data_matrix,user_similarity,type="user")

print("User_prediction__nobias shape:" ,user_prediction_nobias.shape)
print("Item_similarity_nobias shape:" ,item_prediction_nobias.shape)


print(rmse(user_prediction_nobias,test_data_matrix))



print(rmse(item_prediction_nobias,test_data_matrix))


User_prediction__nobias shape: (943, 1682)
Item_similarity_nobias shape: (943, 1682)
3.120061571425749
3.4472573910990616


In [53]:


idx_to_movie = {}
with open('u.item', 'r',encoding = "ISO-8859-1") as f:
    for line in f.readlines():
        info = line.split('|')
        idx_to_movie[int(info[0])-1] = info[1]
        
print(idx_to_movie[1])
print(idx_to_movie[20])

GoldenEye (1995)
Muppet Treasure Island (1996)


In [54]:
def top_k_movies(similarity, mapper, movie_idx, k=6):
    return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]





In [55]:
idx = [0,1,2,20,40] 
for i in idx:
    movies = top_k_movies(item_similarity, idx_to_movie, movie_idx=i)
    names = tuple(movie for movie in movies)
    print("Movie related to\n",idx_to_movie[i],": \n")
    print(names,"\n")

Movie related to
 Toy Story (1995) : 

("Enfer, L' (1994)", 'Here Comes Cookie (1935)', 'Vermont Is For Lovers (1992)', 'Careful (1992)', 'Man from Down Under, The (1943)', 'Daens (1992)') 

Movie related to
 GoldenEye (1995) : 

('Firestorm (1998)', 'Bloody Child, The (1996)', "Gone Fishin' (1997)", 'Thousand Acres, A (1997)', 'Tie That Binds, The (1995)', 'Smile Like Yours, A (1997)') 

Movie related to
 Four Rooms (1995) : 

('Truman Show, The (1998)', 'For the Moment (1994)', 'Open Season (1996)', 'August (1996)', 'Boys (1996)', 'Blood Beach (1981)') 

Movie related to
 Muppet Treasure Island (1996) : 

('Scream of Stone (Schrei aus Stein) (1991)', 'Hotel de Love (1996)', 'Hollow Reed (1996)', 'Losing Chase (1996)', 'Bonheur, Le (1965)', 'Second Jungle Book: Mowgli & Baloo, The (1997)') 

Movie related to
 Billy Madison (1995) : 

('Scream of Stone (Schrei aus Stein) (1991)', 'Designated Mourner, The (1997)', 'Life with Mikey (1993)', 'Love in the Afternoon (1957)', 'Free Willy 3: 

In [56]:
idx_to_user= {}
with open('u.user', 'r',encoding = "ISO-8859-1") as f:
    for line in f.readlines():
        info = line.split('|')
        idx_to_user[int(info[0])-1] = [int(info[0])-1,info[2],info[3]]
print(idx_to_user[1][0])
print(idx_to_user[20])


1
[20, 'M', 'writer']


In [57]:
def top_k_user(similarity, mapper, user_idx, k=6):
    return [mapper[x][0] for x in np.argsort(similarity[user_idx,:])[:-k-1:-1]]



In [59]:
idx = [0,1,2,20,40] 

for i in idx:
    mov=[]
    Users = top_k_user(user_similarity, idx_to_user, user_idx=i)
    
    for u in Users:
        l=np.where(data_matrix[u]==5)[0]
        
    
        for k in l:
            mov.append(idx_to_movie[k])
    print("Suggested movies to the user on basis of user user similarity\n",idx_to_user[i][0],": \n")
    print(mov,"\n")

Suggested movies to the user on basis of user user similarity
 0 : 

['Air Bud (1997)', 'English Patient, The (1996)', 'Wings of the Dove, The (1997)', 'G.I. Jane (1997)', 'Wedding Singer, The (1998)', 'Money Talks (1997)', 'Life Less Ordinary, A (1997)', 'Ma vie en rose (My Life in Pink) (1997)', 'Letter From Death Row, A (1998)', 'English Patient, The (1996)', 'L.A. Confidential (1997)', 'Wings of the Dove, The (1997)', "Eve's Bayou (1997)", 'Sweet Hereafter, The (1997)', 'Four Days in September (1997)', 'Welcome To Sarajevo (1997)', 'English Patient, The (1996)', 'Rosewood (1997)', 'In & Out (1997)', 'Fly Away Home (1996)', 'Mad City (1997)', 'Amistad (1997)', 'Postman, The (1997)', 'Ma vie en rose (My Life in Pink) (1997)', 'Chasing Amy (1997)', 'Big Lebowski, The (1998)', "Ulee's Gold (1997)", 'Ice Storm, The (1997)'] 

Suggested movies to the user on basis of user user similarity
 1 : 

['Searching for Bobby Fischer (1993)', 'Sound of Music, The (1965)', 'Willy Wonka and the Choc